In [3]:
import os
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyOAuth
client_id = SPOTIPY_CLIENT_ID ='bb7c475418484e7784d9cd25b5f9f52c'
client_secret = SPOTIPY_CLIENT_SECRET='b0da0baeeab1499884912aea11f4ca58'
redirect_uri =SPOTIPY_REDIRECT_URI='https://localhost:8080/callback/'


os.environ["SPOTIPY_CLIENT_ID"] = "bb7c475418484e7784d9cd25b5f9f52c"
os.environ["SPOTIPY_CLIENT_SECRET"] = "b0da0baeeab1499884912aea11f4ca58"
os.environ["SPOTIPY_REDIRECT_URI"] = "https://localhost:8080/callback/"

scope = "playlist-modify-public playlist-modify-private user-modify-playback-state user-top-read"
scope +=            " user-modify-playback-state user-read-playback-state user-library-read user-library-modify"
username = 'tonyryanworldwide'

        
token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)        
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope,username=username))


In [3]:
import pandas as pd
from pandas import DataFrame
total = sp.current_user_saved_tracks()['total']
offset = 50
savedtracks = sp.current_user_saved_tracks(limit = 50)['items']
while offset < total:
    
    savedtracks2 =sp.current_user_saved_tracks(limit = 50,offset=offset)['items']
    for track in savedtracks2:
        savedtracks.append(track)
    offset += 50

    
trackinfo ={}   
for trackex in savedtracks:
    id = trackex['track']['id']
    name = trackex['track']['name']
    artistid = trackex['track']['artists'][0]['id']
    seconds = trackex['track']['duration_ms']/1000
    trackinfo[trackex['track']['id']] = {"artist" : artistid,"name":name,"seconds":seconds}
    
trackdf = pd.DataFrame.from_dict(trackinfo,orient = "index").reset_index()
artists = trackdf.artist.unique()
trackdf.columns = ['trackid','artistid','trackname','tracklength_sec']

artistinfo = []
i = 0
while i < len(artists) - 50:
    x=sp.artists(artists[i:i+50])['artists']
    artistinfo.append(x)
    i+=50
artistfinal ={}
for i in artistinfo:
    for j in i:   
        id = j['id']
        name = j['name']
        if len(j['genres']) > 0:
            genre = j['genres']#[0]
        else:
            genre = ''
        artistfinal[id] = {"name":name,"genre": genre}          
artistdf = pd.DataFrame.from_dict(artistfinal,orient = "index").reset_index()
artistdf = artistdf.reset_index()[['index','name','genre']]
artistdf.columns = ['artistid','artistname','genre']
artistdf =artistdf[artistdf['genre']!=''].reset_index()[['artistid','artistname','genre']]

In [4]:
column_names = ["artistid", "artistname", "genre"]

finalartist = pd.DataFrame(columns = column_names)

for i in range(0, len(artistdf)):
    df = artistdf[i:i+1][['artistid','artistname']]
#     print(i)
#     print(df)
    genres = artistdf[i:i+1]['genre'][i]
    gf= DataFrame(genres,columns=['genre'])
    product = (
    df.assign(key=1)
    .merge(gf.assign(key=1), on="key")
    .drop("key", axis=1)
    )
    finalartist = finalartist.append(product)
    

In [30]:
finaltrackinfo = trackdf.merge(finalartist, left_on = 'artistid', right_on = 'artistid', how = 'inner' )
finaltrackinfo['cum_sec'] = finaltrackinfo.groupby('genre')['tracklength_sec'].apply(lambda x: x.cumsum())
finaltrackinfo.to_csv('finaltrackinfo.csv')

In [6]:
genres = finaltrackinfo.groupby('genre').count().reset_index()[['genre','artistid']]
genres.columns=['genre','count']
genres = genres[(genres['count'] >= 25) & (genres['genre']!= '')].sort_values(by ='count', ascending = False)
genrelist = genres['genre'].tolist()
finaltrackinfo = finaltrackinfo[finaltrackinfo['genre'].isin(genres['genre'])]

In [7]:
def genrefilter(df,gen,sec_lim = 36000):
    dfin = df['trackid'][(df['genre'] == gen) & (df['cum_sec'] <= sec_lim)]
    return dfin.tolist()#[0:100]



In [9]:
for gen in genrelist:
    id = sp.user_playlist_create(user=username,name =gen)['id']
    tracks = genrefilter(finaltrackinfo,gen)
#     sp.user_playlist_add_tracks(user = username, playlist_id =id,tracks = tracks,position = 0)
    lengthtracks = len(tracks)
    x= 0
    if lengthtracks <= 100:
        y = lengthtracks
        print("x:{0} y:{1}".format(x,y))
        sp.user_playlist_add_tracks(user = username, playlist_id =id,tracks = tracks[x:y],position = 0)
    else:
        y = 100
        print("1st else: x{} y{}".format(x,y))
        sp.user_playlist_add_tracks(user = username, playlist_id =id,tracks = tracks[x:y],position = 0)
        iteration = 1
        while y >0:
            x = x +100
            if lengthtracks < x  + 100:
                y = lengthtracks   
                print("inif: x{} y{}".format(x,y))
                sp.user_playlist_add_tracks(user = username, playlist_id =id,tracks = tracks[x:y],position = 0)
                y = 0
            else:
                y = y + 100
                print("else x:{} y:{}".format(x,y))
                sp.user_playlist_add_tracks(user = username, playlist_id =id,tracks = tracks[x:y],position = 0)
            iteration += 1

1st else: x0 y100
inif: x100 y138
1st else: x0 y100
inif: x100 y145
1st else: x0 y100
inif: x100 y123
1st else: x0 y100
inif: x100 y155
1st else: x0 y100
inif: x100 y126
1st else: x0 y100
inif: x100 y141
1st else: x0 y100
inif: x100 y141
1st else: x0 y100
inif: x100 y143
1st else: x0 y100
inif: x100 y124
1st else: x0 y100
inif: x100 y136
1st else: x0 y100
inif: x100 y125
1st else: x0 y100
inif: x100 y129
1st else: x0 y100
inif: x100 y110
1st else: x0 y100
inif: x100 y119
1st else: x0 y100
inif: x100 y114
1st else: x0 y100
inif: x100 y110
1st else: x0 y100
inif: x100 y107
1st else: x0 y100
inif: x100 y103
1st else: x0 y100
inif: x100 y102
x:0 y:93
x:0 y:91
x:0 y:91
x:0 y:90
x:0 y:89
x:0 y:83
x:0 y:83
x:0 y:82
x:0 y:77
x:0 y:77
x:0 y:76
x:0 y:75
x:0 y:72
x:0 y:68
x:0 y:66
x:0 y:62
x:0 y:59
x:0 y:59
x:0 y:58
x:0 y:57
x:0 y:56
x:0 y:55
x:0 y:53
x:0 y:53
x:0 y:52
x:0 y:50
x:0 y:50
x:0 y:49
x:0 y:43
x:0 y:43
x:0 y:43
x:0 y:42
x:0 y:42
x:0 y:42
x:0 y:41
x:0 y:40
x:0 y:38
x:0 y:36
x:0 y:35
x:0

In [57]:
ignorelist = ['Your Top Songs 2018','Best Kings']

def playplaylist():
    currentsong = sp.current_playback()
    if currentsong is None:
        print('Nothing''s playing homie')
        return
    currenturi = currentsong['item']['external_urls']['spotify']
    currentsongid = currentsong['item']['id']
    progress_ms = currentsong['progress_ms']
    playlists = sp.user_playlists(username,offset=0)['items']
    playlistbase = (sp.user_playlists(username,offset=50)['items'])
    for pl in playlistbase:
        playlists.append(pl)
    playlistids =[]
    for playlist in playlists:
        playlistid = playlist['id']
        playlistname = playlist['name']
        context_uri = playlist['external_urls']['spotify']
        playlistids.append(playlistid)
        playlist_tracks = sp.playlist_tracks(playlistid)['items']
        for i in range(0,len(playlist_tracks)):
            trackid = playlist_tracks[i]['track']['id']            
            if (trackid == currentsongid):
                print(trackid)
                if playlistname not in ignorelist:
                    print("trackid: {0} context_uri: {1} playlistname: {2} playlistid: {3}".format(trackid,context_uri,playlistname,playlistid))
                    sp.start_playback(context_uri = context_uri,offset ={'position':i},position_ms=progress_ms)
                    return                
    print('No Playlists')
    
playplaylist()


uk americana
symphonic rock
stomp and holler
skate punk
singer-songwriter
roots rock
psychedelic rock
progressive rock
power pop
post-punk
post-hardcore
post-grunge
pop rock
permanent wave
nu metal
new wave
neo mellow
neo classical metal
modern rock
modern folk rock
modern alternative rock
metal guitar
mellow gold
lilith
jam band
irish rock
instrumental rock
indie rock
indie folk
heartland rock
grunge
glam metal
gbvfi
folk-pop
folk rock
country rock
country road
canadian rock
canadian metal
boston rock
blues rock
art rock
alternative rock
alternative pop
alternative metal
album rock
January '21
sexy time
December '20
Your Top Songs 2020
Siddhartha Khosla
permanent wave (2)
November '20
Heartfelt
Alpha Waves
Theta Intuition 5Hz
Beta Waves
Best Instrumental Guitar
Classical Guitar
September '20
Om Shreem Hreem (From Inner Worlds Outer Worlds)
Do Not Play
August '20
Your Top Songs 2018
Best New Found
aaa
lex fridman
The Happiest Days of Our Lives
Generator
Family Mix
Deep Theta 2.0
Dancin

In [49]:
currentsong = sp.current_playback()
print(currentsong)

None


In [35]:
currentsong = sp.current_playback()
currenturi = currentsong['item']['external_urls']['spotify']
trackid = '3BjNm8Zy8rnCSRquLOXSdu'
playlist_tracks = sp.playlist_tracks('3OQAp1crJ3w7ypjdsvUtFS')['items']
for i in range (0,len(playlist_tracks)):
    track = playlist_tracks[i]['track']['id']
    if track == trackid:
        print(i)
        sp.start_playback(context_uri = contexturi,offset={'position':i})

74


In [43]:
currentsong = sp.current_playback()
currentsong['progress_ms']

134924

In [77]:


if (playlistname not in ignorelist):
    print('hi')

In [55]:
# sp.start_playback(context_uri = 'https://open.spotify.com/playlist/3BJ5LAJY99NDCFkxqJtnwo')

playlists[0]['external_urls']['spotify']

'https://open.spotify.com/playlist/3BJ5LAJY99NDCFkxqJtnwo'

In [21]:
currentsongid

# currentsong = sp.current_playback()
# currentson(g['item']['id']

# playlistids[0]

# tracks = sp.playlist_tracks(playlistids[0])['items']
# tracks[0]['track']['id']

'6LcTp8JrfwRweubbBHfCfw'

## For Debugging Delete CreatedPlaylists

In [10]:
# playlists =sp.current_user_playlists(offset=100)['items']
playlists = sp.user_playlists(username,offset=50)['items']
for playlist in playlists:
    name= playlist['name']
    print(name)
    
    id = playlist['uri'][-22:]
    
    if name in genrelist:
        print('deleting{0} id {1}'.format(name,id))
        sp.user_playlist_unfollow(user=username, playlist_id = id)

alternative pop
deletingalternative pop id 71N52aN5QkIFQIre1eFIa2
folk rock
deletingfolk rock id 0E6bdDGWhALf2KW1QD5Eac
power pop
deletingpower pop id 5PPRL4JWW4ttq0DtQ9dOZV
soft rock
deletingsoft rock id 6Jv1kgXehOkrWQXdDE5hb0
roots rock
deletingroots rock id 03rYyy5J0429iOjIlOwuwU
nu metal
deletingnu metal id 1A8GdY548TZVpPUoVi4Zgn
neo mellow
deletingneo mellow id 3CEu9XlCdMbDjsz1Y5cYqw
stomp and holler
deletingstomp and holler id 6WI0RRt59cvCOCbABsutqA
post-grunge
deletingpost-grunge id 1CYwZ4Xo4bRAJfvVc5SkMc
art rock
deletingart rock id 6mVBQdMuPWXyZOviu7PBKg
mellow gold
deletingmellow gold id 3A03xuJDw2q9pKDQS8CUCm
alternative metal
deletingalternative metal id 1iISPtcW7ppovhcKIQk1rC
hip hop
deletinghip hop id 5SLTdyBVmywB8H51GHMEDa
hard rock
deletinghard rock id 0pGz8XC1FzMH4d9lwr6DyJ
permanent wave
deletingpermanent wave id 34mkZOYOJTJcQOrtfdBygK
rap
deletingrap id 6tSILNexSu2GlZKB5cw96v
pop rock
deletingpop rock id 5B3Xup9L0Wr1fmeeOoZd5T
classic rock
deletingclassic rock id 5ze

In [40]:
offset =0
playlistlength =50
while(playlistlength == 50):   
    playlists = sp.user_playlists(username,offset=offset)['items']
    playlistlength = len(playlists)
    print('')
    print("playlistlength {0}".format(playlistlength))
    print('')
    for playlist in playlists:
        name= playlist['name']
#         print(name)
        id = playlist['uri'][-22:]
    
        if name in genrelist:
#             print('')
            print('deleting{0} id {1}'.format(name,id))
            sp.user_playlist_unfollow(user=username, playlist_id = id)
    offset += 50
    


playlistlength 50

deletingfolk-pop id 7mTFQgwiAyvmNYL6FBQat6
deletingfolk rock id 5pFqvN5VXAEp4Ff9Da4wgH
deletingeast coast hip hop id 3DLVkE9y95wCWzcmRZUS7B
deletingdance rock id 22pP6EeQtuj1Q6hfaRGuN9
deletingdance pop id 6zr04DVebTfWSSW01yIDow
deletingcountry rock id 2W7OS8QtwbW7Kw6YVyPqXL
deletingcountry road id 610QGCfBe5U1kWmMeLu1Y0
deletingcountry id 08xioyOlmIXMdCRBcpFYlH
deletingcontemporary country id 6IZnOJw6F8BsnRiEjheVqr
deletingconscious hip hop id 233CUZGuxD1sBRZ1S9PQZt
deletingclassic rock id 374EEgNgrn0AbdCTFcv5nR
deletingclassic canadian rock id 2cmQ7RaX7xPzqiZgOYhZaT
deletingchicago rap id 40SFnKwZOTJN4HIE4OF7zm
deletingcanadian rock id 429NTfKAOvUm4RIBOFINLo
deletingcanadian metal id 30Z96jaSbDiFg4kQmYIuNL
deletingboston rock id 4vQyFnjEQhKyQ4FkfKxpeM
deletingblues rock id 3C4jGazPrdYmvD051ZaVOi
deletingart rock id 38S7N9dC5eMc7iyUPQpL7n
deletingalternative rock id 73T2EdF6cbvlblOzNjA7qu
deletingalternative pop id 6C8kIoXyAGqMB2Pxtd3UWR
deletingalternative metal i

In [27]:
playlists = sp.user_playlists(username,offset=0)['items']
len(playlists)

20